## Этот ноутбук служит для того, чтобы собирать и агрегировать данные для основного файла ***nasdaq_stocks.csv***, 

# который лежит в /Users/denispavlov/Documents/1_strategy/database/nasdaq_stocks.csv

# Результатом исполения этого ноутбука является файл с самыми последними биржевыми котировками, для последующей обработки по стратегии Моментум

# Описание файла:

1. Название файла **nasdaq_stocks.csv**
2. Регулярно обновляется
3. Не перемещаяется и не переименовывается
4. Используется для создания dataframe, которая обрабатывает данные, создаёт и тестирует стратегию Моментум
5. Содержит данные по акциям и ценам закрытия этих акций.

# Исполняемый блок

## Подготовка блокнота к работе

In [1]:
# импорт библиотек
import pandas as pd
import yfinance as yf

In [6]:
# путь к директории где находится файл

import os; os.chdir("..")
os.chdir('/Users/denispavlov/Documents/1_strategy/database/') # см первый текстовый блок
#os.chdir('C:/Users/tomtosov/Desktop')

# Выгрузка данных

Для выгрузки данных необходимо выполнить два основных шага:
1. Выгрузить список (тип series) тикеров, которые торгуются и торговались на бирже(биржах)
2. Выгрузить данные (цены закрытия) по списку из шага 1 из yahoo finance
Для выгрузки списка из шага 1 используем открытые данные с NASDAQ ссылка https://www.nasdaq.com/market-activity/stocks/screener?exchange=NASDAQ&render=download 

**ВАЖНО** скачанный файл переименовывается один раз в nasdaq_stocks.csv и в дальнейшем обновляется только при помощи python

# Шаг 1

Для выгрузки из yahoo finance, нам необходим список акций формата list. Делаем его ниже.

Работаем с файлом nasdaq_stocks.csv ранее скачанным с https://www.nasdaq.com/market-activity/stocks/screener?exchange=NASDAQ&render=download 

In [7]:
# открытие csv-файл через встроенный в pandas reader:
# фактически открываем через создание нового df (nasdaq)


nasdaq = pd.read_csv('nasdaq_stocks.csv')

nasdaq

,Symbol,Name,Last Sale,Net Change,% Change,Market Cap,Country,IPO Year,Volume,Sector,Industry
0,AACG,ATA Creativity Global American Depositary Shares,$1.85,-0.0005,-0.027%,5.845739e+07,China,2008.0,36785,Consumer Discretionary,Service to the Health Industry
1,AACI,Armada Acquisition Corp. I Common Stock,$9.91,0.0000,0.00%,2.052311e+08,United States,2021.0,3,Industrials,Consumer Electronics/Appliances
2,AACIW,Armada Acquisition Corp. I Warrant,$0.17,0.0101,6.316%,0.000000e+00,United States,2021.0,6257,Industrials,Consumer Electronics/Appliances
3,AADI,Aadi Bioscience Inc. Common Stock,$14.22,0.0200,0.141%,2.977949e+08,United States,NaN,72069,Health Care,Biotechnology: Pharmaceutical Preparations
4,AAL,American Airlines Group Inc. Common Stock,$15.02,0.2800,1.90%,9.760686e+09,United States,NaN,23639446,Consumer Discretionary,Air Freight/Delivery Services
...,...,...,...,...,...,...,...,...,...,...,...
4929,ZWRKU,Z-Work Acquisition Corp. Units,$9.87,0.0000,0.00%,0.000000e+00,United States,2021.0,999,Industrials,Consumer Electronics/Appliances
4930,ZWRKW,Z-Work Acquisition Corp. Warrant,$0.1001,-0.0349,-25.852%,0.000000e+00,United States,2021.0,1328,Industrials,Consumer Electronics/Appliances
4931,ZY,Zymergen Inc. Common Stock,$2.85,0.1300,4.779%,2.939512e+08,United States,2021.0,1396863,Industrials,Industrial Specialties
4932,ZYNE,Zynerba Pharmaceuticals Inc. Common Stock,$1.25,0.0300,2.459%,5.449495e+07,United States,2015.0,699243,Health Care,Biotechnology: Pharmaceutical Preparations


In [8]:
# быстрый способ посмотреть кол-во акций. 
# len показывает длинну столбца. поскольку у нас количество строк соответвует количеству акций,
# (индекс=symbols)то мы понимает сколько акций должно быть выгружено

len(nasdaq)

4934

In [9]:
# переменная nasdaq имеет тип DataFrame, а нам нужен тип list, list можно сделать из Series

type(nasdaq)

pandas.core.frame.DataFrame

In [10]:
# меняем тип данных переменной nasdaq с DataFrame на Series
# видим что Length: 4934 = len(nasdaq) 4934, выгрузили правильно!
# переходим к изменению формата для последующей агрегации данных (df)
# выгружаем столбец 'Symbol', так как тикеры, которые мы будем использовать находятся именно в этом стоблце

nasdaq['Symbol']

0        AACG
1        AACI
2       AACIW
3        AADI
4         AAL
        ...  
4929    ZWRKU
4930    ZWRKW
4931       ZY
4932     ZYNE
4933     ZYXI
Name: Symbol, Length: 4934, dtype: object

In [11]:
# проверяем тип данных и видим, что это Series, а в help(yf.download) в tickers указаны типы str и list
# list это отдельный тип данных, поэтому следующий шаг из series в list

type(nasdaq['Symbol'])

pandas.core.series.Series

Биржевые коды для запроса в Yahoo находятся в столбце 'Symbol'. Сохраняем значения столбца в отдельном списке и сделаем запрос (займет намного больше времени)

In [12]:
# переводим столбец Symbol из типа series в тип list

nasdaq_tickers = nasdaq['Symbol'].tolist()

In [13]:
type(nasdaq) #проверка типа (старый  df)

pandas.core.frame.DataFrame

In [14]:
type(nasdaq_tickers) #проверка типа (новый list)

list

In [15]:
[i for i, x in zip(range(len(nasdaq_tickers)), nasdaq_tickers) if not isinstance(x,str)]

[3039]

Не строковая переменная сидит под номером 3039 в списке

In [16]:
nasdaq_tickers[3039]

nan

Это пустое значение, которое сохраненно в float

In [17]:
type(nasdaq_tickers[3039])

float

Как оно выглядит в списке:

In [18]:
nasdaq_tickers[3038:3041]

['MYSZ', nan, 'NAAC']

Удалим его по индексу

In [19]:
del nasdaq_tickers[3039]

Проверяем еще раз значения на не str в списке. Таких больше нет:

In [20]:
[i for i, x in zip(range(len(nasdaq_tickers)), nasdaq_tickers) if not isinstance(x,str)]

[]

Теперь можем делать запрос

In [21]:
nasdaq_stocks = yf.download(tickers = nasdaq_tickers, period = 'max', interval = '1d', group_by = 'ticker')

[                       1%                       ]  44 of 4933 completed

RuntimeError: can't start new thread

[********************  42%                       ]  2055 of 4933 completed